In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import  mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras

#getting data from data folder
input_location = "sample_data/Twitter.csv"

#converting the csv file into a dataframe
data = pd.read_csv(input_location)
data = data.loc[:, ~data.columns.str.contains("^Unnamed")]
print(data.head())

accuracyDict = {}



                                         Id  \
0  6d967b125fcecba6357dbc43f8f380cf2d6d7a51   
1  22dc5f808a8589186767412f39e5c88ae9753d04   
2  7f224e66ad757e3066d45947881bcaab91f8e57f   
3  5acc220e7a337efafc4c847b7f74680ca2515890   
4  863abf834897a06afdc459788b34abeae6298f3a   

                                        post_content sentiment_score  \
0                                  Sana all na lang.               0   
1  キスマイ玉森裕太「ボス恋」台本の裏話明かす\n\n@TBS_asachan @bosskoi...            19.3   
2  SuperM’s album isn’t even out yet and there’s ...               0   
3  மத்திய அரசின் நலத்திட்டங்களை மக்களுக்கு சென்று...               0   
4  【熊本県人吉市】球磨川氾濫後のくま川鉄道の線路被害状況 hamusoku.com/archi...             -20   

  post_length hashtag_count content_url_count tweet_count followers_count  \
0          17             0                 0        1660             133   
1          84             0                 1      318924         1428273   
2          78             0                 0       3

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,17,18,19,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
## Data Cleaning and encoding 
#droping unnecessary feature columns
pd.set_option('max_columns', None)
labels = ['Id', 'post_content', 'published_date_time' ]
data = data.drop(columns = labels)
# data = data.drop([426], axis=0)
le = preprocessing.LabelEncoder()
le.fit(data['media_type'])
data['media_type'] = le.transform(data['media_type'])
print(data.head())

print("hello--------------------------------------------------------------------------------------------")
# data = data.astype(float)
data['sentiment_score'] = data[np.isfinite(pd.to_numeric(data.sentiment_score, errors="coerce"))]
data['post_length'] = data[np.isfinite(pd.to_numeric(data.post_length, errors="coerce"))]
data['hashtag_count'] = data[np.isfinite(pd.to_numeric(data.hashtag_count, errors="coerce"))]
data['content_url_count'] = data[np.isfinite(pd.to_numeric(data.content_url_count, errors="coerce"))]
data['tweet_count'] = data[np.isfinite(pd.to_numeric(data.tweet_count, errors="coerce"))]
data['followers_count'] = data[np.isfinite(pd.to_numeric(data.followers_count, errors="coerce"))]
data['listed_count'] = data[np.isfinite(pd.to_numeric(data.listed_count, errors="coerce"))]
data['mentions_count'] = data[np.isfinite(pd.to_numeric(data.mentions_count, errors="coerce"))]
data['post_author_verified'] = data[np.isfinite(pd.to_numeric(data.post_author_verified, errors="coerce"))]
data['likes'] = data[np.isfinite(pd.to_numeric(data.likes, errors="coerce"))]
data['shares'] = data[np.isfinite(pd.to_numeric(data.shares, errors="coerce"))]
data['comments'] = data[np.isfinite(pd.to_numeric(data.comments, errors="coerce"))]
data['impact'] = data[np.isfinite(pd.to_numeric(data.impact, errors="coerce"))]

data['sentiment_score'] = data['sentiment_score'].astype(float)
data['post_length'] = data['post_length'].astype(float)
data['hashtag_count'] = data['hashtag_count'].astype(float)
data['content_url_count'] = data['content_url_count'].astype(float)
data['tweet_count'] = data['tweet_count'].astype(float)
data['followers_count'] = data['followers_count'].astype(float)
data['mentions_count'] = data['mentions_count'].astype(float)
data['post_author_verified'] = data['post_author_verified'].astype(float)
data['likes'] = data['likes'].astype(float)
data['shares'] = data['shares'].astype(float)
data['comments'] = data['comments'].astype(float)
data['impact'] = data['impact'].astype(float)
data['listed_count'] = data['listed_count'].astype(float)


# print(data.index[[426]])
# print(data.index[[427]])
# Encoding non numeric data
le = preprocessing.LabelEncoder()
le.fit(data['media_type'])
data['media_type'] = le.transform(data['media_type'])
print(data['sentiment_score'].head())
print(type(data.loc[650, 'sentiment_score']))
print(type(data.loc[651, 'sentiment_score']))
print(type(data.loc[652, 'sentiment_score']))
print(type(data.loc[652, 'post_length']))
print(type(data.loc[652, 'hashtag_count']))
print(type(data.loc[652, 'content_url_count']))
# for col in data:
#   i=0
#   for row in data[col]:
#     if type(row)== type('string'):
#       data.loc[i, 'sentiment_score'] = 0
#     i+=1

print("hello--------------------------------------------------------------------------------------------")
print(data['sentiment_score'].head())
print(data.loc[650, 'sentiment_score'])

  sentiment_score post_length hashtag_count content_url_count tweet_count  \
0               0          17             0                 0        1660   
1            19.3          84             0                 1      318924   
2               0          78             0                 0       39976   
3               0         284             2                 0        8045   
4             -20          80             0                 1      210996   

  followers_count listed_count  media_type mentions_count  \
0             133            0         402              0   
1         1428273         5454         402              0   
2            1560            0         402              0   
3           27191           26         402              0   
4            2385            0         401              0   

  post_author_verified likes shares comments impact  
0                    0     0      0        0      0  
1                    1    10      1        0    110  
2       

In [ ]:
## Cleaning up NaN Values
print(data.isna().sum())
default_val = 0.0
for feature in data:
  print(feature)
  data[feature] = data[feature].fillna(default_val, axis=0)

print(data.head(10))

sentiment_score         586
post_length             586
hashtag_count           586
content_url_count       586
tweet_count             586
followers_count         586
listed_count            586
media_type                0
mentions_count          588
post_author_verified    586
likes                   586
shares                  586
comments                586
impact                  586
dtype: int64
sentiment_score
post_length
hashtag_count
content_url_count
tweet_count
followers_count
listed_count
media_type
mentions_count
post_author_verified
likes
shares
comments
impact
   sentiment_score  post_length  hashtag_count  content_url_count  \
0             0.00         0.00           0.00               0.00   
1            19.30        19.30          19.30              19.30   
2             0.00         0.00           0.00               0.00   
3             0.00         0.00           0.00               0.00   
4           -20.00       -20.00         -20.00             -20.00   
5   

In [ ]:
## Data Scalaing
#Scaling data
scale_cols = ['sentiment_score', 'post_length', 'tweet_count', 'followers_count', 'listed_count', 'mentions_count', 'likes', 'shares', 'comments', 'impact']
mms = preprocessing.MinMaxScaler()
mms.fit(data[scale_cols])
data[scale_cols] = mms.transform(data[scale_cols])
print(data.head())
print(data['media_type'].head())



   sentiment_score  post_length  hashtag_count  content_url_count  \
0         0.013158     0.013158            0.0                0.0   
1         0.025855     0.025855           19.3               19.3   
2         0.013158     0.013158            0.0                0.0   
3         0.013158     0.013158            0.0                0.0   
4         0.000000     0.000000          -20.0              -20.0   

   tweet_count  followers_count  listed_count  media_type  mentions_count  \
0     0.013158         0.013158      0.013158         402          0.5000   
1     0.025855         0.025855      0.025855         402          0.9825   
2     0.013158         0.013158      0.013158         402          0.5000   
3     0.013158         0.013158      0.013158         402          0.5000   
4     0.000000         0.000000      0.000000         401          0.0000   

   post_author_verified     likes    shares  comments    impact  
0                   0.0  0.013158  0.013158  0.013158  0

In [ ]:
#dividing the data into dependent and independent variables

#splitting the dataset into train and test splits
X = data.drop(columns = 'impact') 
y = data['impact']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)
print(y_train.head())

41688    0.025263
18456    0.013158
18962    0.026112
35005    0.021691
29685    0.013158
Name: impact, dtype: float64


In [ ]:
## Getting Accuracy score
def evalModel(model, y_test, y_pred_class):
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_class))
    r2 = r2_score(y_test, y_pred_class)
    # conmat = metrics.confusion_matrix(y_test, y_pred_class)
    # sns.heatmap(conmat, annot=True)
    # plt.title("Confusion Matrix" + str(model))
    # plt.xlabel("predicted")
    # plt.ylabel("actual")
    # plt.show()
    return rmse, r2;

In [ ]:
# Decision Tree Model
ts1 = time.time()
print("Decision Tree model")
DT = DecisionTreeRegressor(max_depth=12, random_state=0)
DT.fit(X_train, y_train)
y_pred_class = DT.predict(X_test)
accuracy = evalModel(DT, y_test, y_pred_class)
accuracyDict['Decision Tree'] = accuracy
te1 = time.time()
print("Total time taken: ", te1-ts1)
print(accuracyDict)
print(y_pred_class)

# Decision Tree with GridSearch tuning
# print("\nTuning Decision Tree model with GridSearchCV\n")
# param_grid = {'criterion':['mse','friedman_mse', 'mae','poisson'],
#                 'max_depth':[3,5,7,9,11,13,15],
#                 'min_samples_split':[2,3,4,5,6,7,8],
#                 'random_state':[0]}
# grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, n_jobs=-1,  cv=500)
# grid_search.fit(X_train,y_train)
# print("Best param_grids: ", grid_search.best_params )
# print("Best cross-validation score: ", grid_search.best_score *100, "%")
# print("Best estimator: ", grid_search.best_estimator )
# dt = grid_search.best_estimator 
# y_pred_class = dt.predict(X_test)
# accuracy = evalModel(dt, y_test, y_pred__class)
# accuracyDict['Decision Tree tuning GridSearchCV'] = accuracy 
# print(accuracyDict)

Decision Tree model
Total time taken:  -0.08078765869140625
{'Decision Tree': (0.008791189250671214, 0.3805235410691038)}
[0.01312583 0.01312583 0.02615474 ... 0.01312583 0.00516359 0.00516359]


In [ ]:
## Linear Regression Model
ts2 = time.time()
print("Linear Regression")
LR = LinearRegression()
LR.fit(X_train, y_train)
y_pred_class = LR.predict(X_test)
# print('Coefficient: ', LR.coef_)
# print("Mean Squared error: ", np.mean(LR.predict(X_test)- y_test) ** 2)
# print("Variance Score: ", LR.score(X_test, y_test))
accuracy = evalModel(LR, y_test, y_pred_class)
accuracyDict['Linear Regression'] = accuracy 
te2 = time.time()
print("Total time taken: ", te2- ts2)
print(accuracyDict)
print(y_pred_class)

Linear Regression
Total time taken:  0.024744272232055664
{'Decision Tree': (0.008791189250671214, 0.3805235410691038), 'Linear Regression': (8.034709755489361e-17, 1.0)}
[0.01315789 0.01315789 0.02596053 ... 0.01315789 0.00515789 0.00555263]


In [ ]:
## Deep Neural Network
print("DNN")
ts3 = time.time()
#converting dataframes to tensors
Sentiment_score = tf.feature_column.numeric_column('sentiment_score')
Post_Length= tf.feature_column.numeric_column('post_length')
Hashtag_count= tf.feature_column.numeric_column('hashtag_count')
Content_URL_count= tf.feature_column.numeric_column('content_url_count')
Tweet_count= tf.feature_column.numeric_column('tweet_count')
Followers_count= tf.feature_column.numeric_column('followers_count')
Listed_Count= tf.feature_column.numeric_column('listed_count')
Media_Type= tf.feature_column.numeric_column('media_type')
Mentions_Count= tf.feature_column.numeric_column('mentions_count')
Post_author_verified= tf.feature_column.numeric_column('post_author_verified')
Likes= tf.feature_column.numeric_column('likes')
Shares= tf.feature_column.numeric_column('shares')
Comments= tf.feature_column.numeric_column('comments')
Impact= tf.feature_column.numeric_column('impact')

#defining feature columns
feature_columns = [Sentiment_score,
Post_Length,
Hashtag_count,
Content_URL_count,
Tweet_count,
Followers_count,
Listed_Count,
Media_Type,
Mentions_Count,
Post_author_verified,
Likes,
Shares,
Comments]

#training input method
def train_input_fn(features, labels, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  
  return dataset.shuffle(1000).repeat().batch(batch_size)

#evaluation input method
def eval_input_fn(features, labels, batch_size):
  if labels is None:
      inputs = dict(features)
  else:
      inputs = (dict(features), labels)

  dataset = tf.data.Dataset.from_tensor_slices(inputs)

  assert batch_size is not None, "batch_size must not be None"
  dataset = dataset.batch(batch_size)

  return dataset


# tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0)
dnn = tf.estimator.DNNRegressor(feature_columns=feature_columns, hidden_units=[150, 500, 100], optimizer= tf.keras.optimizers.Adagrad(learning_rate=0.001), batch_norm=True )
#training the model
dnn.train(input_fn=lambda: train_input_fn(X_train, y_train, 500), steps=1000 )
#evaluating the accuracy of the model
result = dnn.evaluate(input_fn=lambda: eval_input_fn(X_test, y_test, 500) )
y_pred_class = []
#model.predict() function takes the same input function as evaluate
#the key class_ids stores the corresponding value of the prediciton the model made
timeout= time.time() + 2
for single_prediction in dnn.predict(input_fn=lambda: eval_input_fn(X_test,labels=None, batch_size= 500)):
    for key, value in single_prediction.items():
            y_pred_class.append(value[0])

print('hello----------------------------------------------------------------------------------------------')
print(y_pred_class)
print(y_test)
accuracy = evalModel("DNN", y_test, y_pred_class)
te3 = time.time()
print("Total time taken: ", te3 - ts3)
accuracyDict['Dnn'] = accuracy
print(accuracyDict)

DNN
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpq_h5uo_o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO